In [1]:
#Basic packages always been used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Data preprocessing useful function
from sklearn.preprocessing import LabelEncoder
# Function for spilting training & testing data set
from sklearn.model_selection import train_test_split
# ALgorithm
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
# Functions for evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve, roc_auc_score, roc_curve

df = pd.read_csv("heart.csv")

In [2]:
new_df=df.dropna(axis=0, how='any').copy()
new_df.describe(include='all')

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,918.000000,918,918,918.000000,918.000000,918.000000,918,918.000000,918,918.000000,918,918.000000
unique,NaN,2,4,NaN,NaN,NaN,3,NaN,2,NaN,3,NaN
top,NaN,M,ASY,NaN,NaN,NaN,Normal,NaN,N,NaN,Flat,NaN
freq,NaN,725,496,NaN,NaN,NaN,552,NaN,547,NaN,460,NaN
mean,53.510893,NaN,NaN,132.540305,240.581699,0.233115,NaN,136.809368,NaN,0.887364,NaN,0.553377
std,9.432617,NaN,NaN,17.989941,53.982967,0.423046,NaN,25.460334,NaN,1.066570,NaN,0.497414
min,28.000000,NaN,NaN,80.000000,85.000000,0.000000,NaN,60.000000,NaN,-2.600000,NaN,0.000000
25%,47.000000,NaN,NaN,120.000000,214.000000,0.000000,NaN,120.000000,NaN,0.000000,NaN,0.000000
50%,54.000000,NaN,NaN,130.000000,223.000000,0.000000,NaN,138.000000,NaN,0.600000,NaN,1.000000
75%,60.000000,NaN,NaN,140.000000,267.000000,0.000000,NaN,156.000000,NaN,1.500000,NaN,1.000000


In [3]:
print('Age',new_df['Age'].unique())
print('Sex',new_df['Sex'].unique())
print('ChestPainType',new_df['ChestPainType'].unique())
print('RestingBP',new_df['RestingBP'].unique())
print('Cholesterol',new_df['Cholesterol'].unique())
print('FastingBS',new_df['FastingBS'].unique())
print('RestingECG',new_df['RestingECG'].unique())
print('MaxHR',new_df['MaxHR'].unique())
print('ExerciseAngina',new_df['ExerciseAngina'].unique())
print('Oldpeak',new_df['Oldpeak'].unique())
print('ST_Slope',new_df['ST_Slope'].unique())
print('HeartDisease',new_df['HeartDisease'].unique())

Age [40 49 37 48 54 39 45 58 42 38 43 60 36 44 53 52 51 56 41 32 65 35 59 50
 47 31 46 57 55 63 66 34 33 61 29 62 28 30 74 68 72 64 69 67 73 70 77 75
 76 71]
Sex ['M' 'F']
ChestPainType ['ATA' 'NAP' 'ASY' 'TA']
RestingBP [140 160 130 138 150 120 110 136 115 100 124 113 125 145 112 132 118 170
 142 190 135 180 108 155 128 106  92 200 122  98 105 133  95  80 137 185
 165 126 152 116 144 154 134 104 139 131 141 178 146 158 123 102  96 143
 172 156 114 127 101 174  94 148 117 192 129 164]
Cholesterol [289 180 283 214 195 339 237 208 207 284 211 164 204 234 273 196 201 248
 267 223 184 288 215 209 260 468 188 518 167 224 172 186 254 306 250 177
 227 230 294 264 259 175 318 216 340 233 205 245 194 270 213 365 342 253
 277 202 297 225 246 412 265 182 218 268 163 529 100 206 238 139 263 291
 229 307 210 329 147  85 269 275 179 392 466 129 241 255 276 282 338 160
 156 272 240 393 161 228 292 388 166 247 331 341 243 279 198 249 168 603
 159 190 185 290 212 231 222 235 320 187 266 287 404 312 251

In [4]:
df_mapping=new_df.copy()

map_Sex={'M':0, 'F':1}
map_ChestPainType={'ATA':0, 'NAP':1, 'ASY':2, 'TA':3}
map_RestingECG={'Normal':0, 'ST':1, 'LVH':2}
map_ExerciseAngina={'N':0, 'Y':1}
map_ST_Slope={'Up':0, 'Flat':1, 'Down':1}

df_mapping['Sex']=df_mapping['Sex'].map(map_Sex)
df_mapping['ChestPainType']=df_mapping['ChestPainType'].map(map_ChestPainType)
df_mapping['RestingECG']=df_mapping['RestingECG'].map(map_RestingECG)
df_mapping['ExerciseAngina']=df_mapping['ExerciseAngina'].map(map_ExerciseAngina)
df_mapping['ST_Slope']=df_mapping['ST_Slope'].map(map_ST_Slope)

In [5]:
dummied_new_df=pd.get_dummies(new_df, columns=['Sex', 'ChestPainType', 'RestingECG',
                                               'ExerciseAngina', 'ST_Slope'])

In [6]:
x=dummied_new_df.drop(['HeartDisease'],axis=1).copy()
y=df['HeartDisease'].copy()

In [7]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)

In [13]:
clf1 = SVC(kernel = 'rbf', random_state=100)
param_grid1 = [{'C': [1,5,10],
                'gamma':[0.01,0.05,0.1],
               }]
cross_val=KFold(n_splits=3,shuffle=True,random_state=4)

In [14]:
# using gridsearch for hyper-parameter tuning
specificity = make_scorer(recall_score, pos_label=0)
gridS_model = GridSearchCV(estimator=clf1,
                                param_grid=param_grid1,
                                scoring='accuracy',
                                n_jobs=-1,
                                cv=cross_val,
                                verbose=0,
                                refit=True)

cv_outer = KFold(n_splits=3, shuffle=True, random_state=1)

train_scores = cross_val_score(gridS_model, x_train, y_train, scoring='accuracy', cv=cv_outer, n_jobs=-1)
train_scores_f1 = cross_val_score(gridS_model, x_train, y_train, scoring='f1', cv=cv_outer, n_jobs=-1)
train_scores_precision = cross_val_score(gridS_model, x_train, y_train, scoring='precision', cv=cv_outer, n_jobs=-1)
train_scores_sen = cross_val_score(gridS_model, x_train, y_train, scoring='recall', cv=cv_outer, n_jobs=-1)
train_scores_spe = cross_val_score(gridS_model, x_train, y_train, scoring=specificity, cv=cv_outer, n_jobs=-1)

test_scores = cross_val_score(gridS_model, x_test, y_test, scoring='accuracy', cv=cv_outer, n_jobs=-1)
test_scores_f1 = cross_val_score(gridS_model, x_test, y_test, scoring='f1', cv=cv_outer, n_jobs=-1)
test_scores_precision = cross_val_score(gridS_model, x_test, y_test, scoring='precision', cv=cv_outer, n_jobs=-1)
test_scores_sen = cross_val_score(gridS_model, x_test, y_test, scoring='recall', cv=cv_outer, n_jobs=-1)
test_scores_spe = cross_val_score(gridS_model, x_train, y_train, scoring=specificity, cv=cv_outer, n_jobs=-1)

#peforming gridsearch & save each models training result
print('train ACC:',round((np.mean(train_scores))*100,2),'+/-',round((np.std(train_scores))*100,2))
print('train fls:',round((np.mean(train_scores_f1))*100,2),'+/-',round((np.std(train_scores_f1))*100,2))
print('train precision:',round((np.mean(train_scores_precision))*100,2),'+/-',round((np.std(train_scores_precision))*100,2))
print('train sen:',round((np.mean(train_scores_sen))*100,2),'+/-',round((np.std(train_scores_sen))*100,2))
print('train spe:',round((np.mean(train_scores_spe))*100,2),'+/-',round((np.std(train_scores_spe))*100,2))
print('+----------------------------------------+')

print('test ACC:',round((np.mean(test_scores))*100,2),'+/-',round((np.std(test_scores))*100,2))
print('test fls:',round((np.mean(test_scores_f1))*100,2),'+/-',round((np.std(test_scores_f1))*100,2))
print('test precision:',round((np.mean(test_scores_precision))*100,2),'+/-',round((np.std(test_scores_precision))*100,2))
print('test sen:',round((np.mean(test_scores_sen))*100,2),'+/-',round((np.std(test_scores_sen))*100,2))
print('test sen:',round((np.mean(test_scores_spe))*100,2),'+/-',round((np.std(test_scores_spe))*100,2))

train ACC: 65.93 +/- 3.45
train fls: 71.72 +/- 1.65
train precision: 66.42 +/- 4.83
train sen: 78.49 +/- 3.25
train spe: 51.0 +/- 10.63
+----------------------------------------+
test ACC: 60.83 +/- 5.07
test fls: 71.51 +/- 4.38
test precision: 61.21 +/- 6.89
test sen: 86.76 +/- 2.94
test sen: 51.0 +/- 10.63
